# Imports

In [ ]:
import openai
import pandas as pd
import json
import string
import numpy as np
import time
from openai.embeddings_utils import pca_components_from_embeddings
import sys

from causalnex.structure import StructureModel
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
def request_gpt(prompt: str = '', content: str = ''):
    """
    Wrapper that calls OpenAI's API to generate a response based on a system prompt and a body of text.

    system_prompt: str
    body: str

    return: str
    """
    _body = prompt + content
    res = openai.ChatCompletion.create(
        model='gpt-4',
        messages=[{'role': 'user', 'content': _body}]
    )
    return res.choices[0].message.content

In [4]:
# Load "credentials.json" which contains the value "open_ai_key"
openai.api_key = json.load(open("../credentials.json"))["open_ai_key"]

In [ ]:
# Load the data

corpus = pd.read_csv('CORPUS_ICPB.csv', index_col=0)
corpus

# Pre-processing

In [5]:
### Pre-processing of the data

# Drop some columns (modifyAt, createAt, idArticle, vecteur) which cannot be used
df = corpus[["titre", "source", "texte", "date", "auteur"]]

# Drop duplicated rows
df = df.drop_duplicates()

# Remove NaN values (In this case we remove a row if any NaN in it)
df = df.dropna()

# Create the column content which is the aggregation of title and text
df["content"] = "Titre: " + df["titre"]+ "\nContenu: " + df["texte"]

# Reset index
df.reset_index(drop=True, inplace=True)

# Save
df.to_csv('dataset_preprocessed.csv', index=False)

# Financial data filter

In [20]:
# Setup
FILTER_PROMPT = """

Write 1 if the following article is about finance or can impact the stock market, else 0:

--

"""

i_finance = 0 # Index of the row to process
df['is_finance'] = None # Creation of a new column which is True if related to Finance, else False
n_limit_finance = 1000 # Number of rows 

In [ ]:
# Creation of a new column which is True if related to Finance, else False
while i_finance<n_limit_finance and i_finance<len(df):
    try:
        print("Request on the {}th row".format(i_finance))
        content = df.content.iloc[i_finance]
        res = request_gpt(prompt=FILTER_PROMPT, content=content)
        _bool = bool(int(res))
        df['is_finance'][i_finance] = _bool
        i_finance+=1
    except Exception as e:
        print("Something went wrong:\n\n" + str(e))
        time.sleep(2)

In [195]:
df_finance = df[df.is_finance == True].reset_index(drop=True)
df_finance.to_csv("dataset_finance.csv", index=False)
df_finance

,titre,source,texte,date,auteur,content,is_finance
0,Grève du 31 janvier : à quelles perturbations ...,LePoint.fr,"La première journée de grève, jeudi 19 janvier...",2023-01-25 14:46:00+00:00,Par Thibaut Déléaz,Titre: Grève du 31 janvier : à quelles perturb...,True
1,"Chômage : une baisse de 3,6 % au quatrième tri...",LePoint.fr,"Le nombre de chômeurs en France s'élève à à 3,...",2023-01-25 13:24:00+00:00,Source AFP,"Titre: Chômage : une baisse de 3,6 % au quatri...",True
2,Le Vatican définit l’investissement chrétien e...,Challenges,"LES MARCHÉS AUJOURD'HUI\nPX1\n▼ -0,54%\nPX4\n▼...",2022-05-12 17:21:08+00:00,Article de Pierre,Titre: Le Vatican définit l’investissement chr...,True
3,Surtaxe d’habitation : découvrez les villes qu...,Capital,"LES MARCHÉS AUJOURD'HUI\nPX1\n▼ -1,44%\nPX4\n▼...",2022-12-16 16:36:45+00:00,Article de Jean,Titre: Surtaxe d’habitation : découvrez les vi...,True
4,Intergénérationnel : Focus sur les nombreux pr...,Zinfos,"LES MARCHÉS AUJOURD'HUI\nPX1\n▲ +0,60%\nPX4\n▲...",2022-12-19 16:40:40+00:00,Article de Ville de Saint,Titre: Intergénérationnel : Focus sur les nomb...,True
...,...,...,...,...,...,...,...
382,"Assemblée nationale : Cyrielle Chatelain, l’in...",20 Minutes,La jeune présidente du groupe écologiste à l’A...,2023-02-19 12:25:00+00:00,Rachel Garrat Valcarcel,Titre: Assemblée nationale : Cyrielle Chatelai...,True
383,L'Ukraine tente de réparer ses sites énergétiq...,Euronews français,Les autorités ukrainiennes tentent d'atténuer ...,2023-02-20 08:47:32+00:00,Euronews,Titre: L'Ukraine tente de réparer ses sites én...,True
384,"Avec la flambée des prix de l'énergie, le chif...",La Tribune,L'avenir s'annonce radieux pour Air Liquide qu...,2023-02-20 08:47:32+00:00,latribune.fr,"Titre: Avec la flambée des prix de l'énergie, ...",True
385,Guerre en Ukraine : des démocrates américains ...,L'Express,Le président américain fait face à des critiqu...,2023-02-20 08:47:32+00:00,lexpress.fr,Titre: Guerre en Ukraine : des démocrates amér...,True


# Extraction of variables

In [7]:
# Setup
EXTRACTION_PROMPT = """

Mon objectif est d'identifier et d'établir des relations causales à partir d'un vaste corpus de comptes rendus boursiers. Il s'agit de comprendre comment différents événements, tels que les annonces financières, les décisions politiques ou les fluctuations économiques, influencent les marchés financiers.
Pour cela, j'ai besoin d'identifier et d'extraire les variables de traitement, de résultat et de confusion de l'article.

Ta mission est de trouver ces variables pertinentes dans un article de finance. Elles appartiendront à une ou plusieurs catégories parmis :

[Transport et Infrastructure, Éducation et Jeunesse, Environnement, Économie, Entreprise, Social, Identités et Personnalités, Politique et Géopolitique,  International, Condition de vie, Justice et Loi].

Tu dois les classer dans ces catégories. Tu peux classer une variable pertinente dans plusieurs catégories.

Je voudrais en plus que tu repères une tendance dans l'article qui sera la variable de résultat que l'on nommera "Indicateur". Cet "Indicateur" sera un terme parmis "POSITIF", "NEUTRE" ou "NEGATIF" en fonction du ressenti de l'article sur les cours et indices boursiers, l'évolution des marchés...
Par exemple, si tu vois :\n
Lorsqu'une entreprise connaît une baisse significative de ses bénéfices ou une réduction de son activité, elle peut être amenée à réduire ses effectifs en effectuant des licenciements ou en gelant les embauches, ce qui peut contribuer à une augmentation du taux de chômage dans la région où elle opère. \n
Je voudrais que tu arrives à extraire : Situation Financière de l'Entreprise, Décisions de l'Entreprise, Taux de Chômage, Cours Boursier de l'Entreprise 

Tu dois retourner uniquement une chaîne de caractères au format JSON suivant:
{
    "Indicateur": ["valeur0"],
    "Transport et Infrastructure" : [],
    "Éducation et Jeunesse": [],
    "Environnement": [],
    "Économie": [Taux de Chômage, Cours Boursier de l'Entreprise],
    "Entreprise": [Situation Financière de l'Entreprise, Décisions de l'Entreprise, Taux de Chômage, Cours Boursier de l'Entreprise],
    "Social": [Taux de Chômage],
    "Identités et Personnalités": [],
    "Politique et Géopolitique": [],
    "International": [],
    "Condition de vie": [Taux de Chômage],
    "Justice et Loi": []
    ...
}
où les valeurs sont les mots repérés dans l'article en lien avec la variable extraite.

Voici l'article à analyser:

"""

i_extraction = 0 # Index of the row to process
df_finance['extraction'] = None # Creation of a new column "extraction" which contains a dict. keys are variables and values are text from the article related to the variable

n_limit_extraction = 400

In [ ]:
while i_extraction<n_limit_extraction and i_extraction<len(df_finance):
    try:
        print("Request on the {}th row".format(i_extraction))
        content = df_finance.content.iloc[i_extraction]
        res = request_gpt(prompt=EXTRACTION_PROMPT, content=content)
        _json = json.loads(res.replace("\n", ""))
        df_finance['extraction'][i_extraction] = _json
        i_extraction+=1
    except Exception as e:
        print("Something went wrong:\n\n" + str(e))
        time.sleep(5)

In [26]:
df_finance = df_finance[~df_finance["extraction"].isna()]
df_finance.to_csv("dataset_extract.csv", index=False)
df_finance

,titre,source,texte,date,auteur,content,is_finance,extraction
0,Grève du 31 janvier : à quelles perturbations ...,LePoint.fr,"La première journée de grève, jeudi 19 janvier...",2023-01-25 14:46:00+00:00,Par Thibaut Déléaz,Titre: Grève du 31 janvier : à quelles perturb...,True,"{'Indicateur': ['NEGATIF'], 'Transport et Infr..."
1,"Chômage : une baisse de 3,6 % au quatrième tri...",LePoint.fr,"Le nombre de chômeurs en France s'élève à à 3,...",2023-01-25 13:24:00+00:00,Source AFP,"Titre: Chômage : une baisse de 3,6 % au quatri...",True,"{'Indicateur': ['POSITIF'], 'Transport et Infr..."
2,Le Vatican définit l’investissement chrétien e...,Challenges,"LES MARCHÉS AUJOURD'HUI\nPX1\n▼ -0,54%\nPX4\n▼...",2022-05-12 17:21:08+00:00,Article de Pierre,Titre: Le Vatican définit l’investissement chr...,True,"{'Indicateur': ['NEGATIF'], 'Transport et Infr..."
3,Surtaxe d’habitation : découvrez les villes qu...,Capital,"LES MARCHÉS AUJOURD'HUI\nPX1\n▼ -1,44%\nPX4\n▼...",2022-12-16 16:36:45+00:00,Article de Jean,Titre: Surtaxe d’habitation : découvrez les vi...,True,"{'Indicateur': ['NEGATIF'], 'Transport et Infr..."
4,Intergénérationnel : Focus sur les nombreux pr...,Zinfos,"LES MARCHÉS AUJOURD'HUI\nPX1\n▲ +0,60%\nPX4\n▲...",2022-12-19 16:40:40+00:00,Article de Ville de Saint,Titre: Intergénérationnel : Focus sur les nomb...,True,"{'Indicateur': ['POSITIF'], 'Transport et Infr..."
...,...,...,...,...,...,...,...,...
163,Ukraine: la stratégie russe incertaine concern...,RFI,"À Kherson, les autorités pro-russes annoncent ...",2022-08-11 14:39:45+00:00,Léo Vidal Giraud,Titre: Ukraine: la stratégie russe incertaine ...,True,"{'Indicateur': ['NEGATIF'], 'Transport et Infr..."
164,Les pays du Sud doivent trouver 2.400 milliard...,La Tribune,Ils sont les moins responsables des émissions ...,2022-08-11 14:47:40+00:00,latribune.fr,Titre: Les pays du Sud doivent trouver 2.400 m...,True,"{'Indicateur': ['NEUTRE'], 'Transport et Infra..."
165,"Climat : ""L'industrie, c'est 20% des émissions...",franceinfo,"""Quand on parle de l'industrie française, on a...",2022-08-11 14:48:19+00:00,Franceinfo,"Titre: Climat : ""L'industrie, c'est 20% des ém...",True,"{'Indicateur': ['NEUTRE'], 'Transport et Infra..."
166,COP27 : l’aide aux pays du Sud au centre des d...,Euronews français,🌍 Les dirigeants des pays en développement réc...,2022-08-11 14:48:51+00:00,euronews,Titre: COP27 : l’aide aux pays du Sud au centr...,True,"{'Indicateur': ['NEUTRE'], 'Transport et Infra..."


# Creation of the new dataframe where columns are variables

In [27]:
# Creation of the new dataframe where df[i,j] are the text from article i related to variable j
df_variables = pd.DataFrame()
for row in df_finance.iterrows():
    _dict = row[1].extraction
    if _dict is None:
        break
    new_dict = {}
    for key, value in _dict.items():
        if type(value)==list:
            value = " ".join(value)
        new_dict[key] = value
    df_variables = pd.concat([df_variables, pd.DataFrame({row[0]: new_dict}).transpose()])

df_variables.to_csv("dataset_variable.csv", index=False)
df_variables

,Condition de vie,Entreprise,Environnement,Identités et Personnalités,Indicateur,International,Justice et Loi,Politique et Géopolitique,Social,Transport et Infrastructure,Économie,Éducation et Jeunesse,Santé
0,usagers grève vacances scolaires zone A,SNCF CGT Cheminots SUD-Rail FO transports,,exécutif Assemblée nationale ministère de l'In...,NEGATIF,,projet de réforme des retraites Assemblée nati...,manifestants projet de réforme des retraites s...,grève projet de réforme des retraites ministèr...,transports en commun et train SNCF CGT Chemino...,projet de réforme des retraites pavé dans tout...,vacances scolaires zone A grève,NaN
1,Chômage,,,,POSITIF,,,Ministère du travail,Chômage Tendance du chômage Ministère du trava...,,Chômage Tendance du chômage Ministère du trava...,,NaN
2,,secteurs à exclure responsabilité sociétale de...,normes environnementales écologie,pape François cardinal Turkson Académie pontif...,NEGATIF,Vatican pays à exclure,normes d'investissement responsable directive ...,Commission européenne normes internationales,"critères environnementaux, sociaux et de gouve...",,investissement responsable investissements nor...,,NaN
3,Taxe d’habitation Condition d’habitation Commu...,,,Direction générale des finances publiques (DGF...,NEGATIF,,Loi de finances Zonage,Loi de finances,Condition d’habitation Communes Résidences sec...,,Taxe d’habitation Marché immobilier Zonage Loi...,,NaN
4,Développement des actions intergénérationnelle...,,,Roxanne PAUSÉ DAMOUR,POSITIF,,,Politique Sénior Développement des actions int...,Politique Sénior Développements des actions in...,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Evacuation des civils Electricité,,,Autorités russes Autorités pro-russes Chef de ...,NEGATIF,Ukraine Russie Moscou Kiev Occidentaux,,Stratégie russe Autorités russes Propagande ru...,Evacuation des civils Electricité,Evacuation des civils Electricité,,,NaN
164,Emissions de gaz à effet de serre Réchauffemen...,Investissements Investisseurs privés et publics,Action climatique Emissions de gaz à effet de ...,Vera Songwe Nicholas Stern Amar Bhattacharya,NEUTRE,Pays du sud Pays émergents et en développement...,,27e conférence internationale sur le climat Pa...,,,Financements climatiques Investissements Pays ...,,NaN
165,,Effort climatique Emissions de gaz à effet de ...,Emissions de gaz à effet de serre Décarbonatio...,Roland Lescure,NEUTRE,Grand pays industriel,,Effort climatique Emissions de gaz à effet de ...,,,,,NaN
166,conséquences du réchauffement climatique,,changement climatique réchauffement climatique...,président du Zimbabwe Emmerson Mnangagwa prési...,NEUTRE,COP27 Égypte Afrique Barbade,promesse de financement aux pays du Sud,dirigeants africains pays en développement pay...,conséquences du réchauffement climatique,,financement action climatique transition énerg...,,NaN


In [33]:
(~(df_variables.isna()) | (df_variables=="")).sum()

Condition de vie               168
Entreprise                     168
Environnement                  167
Identités et Personnalités     168
Indicateur                     168
International                  168
Justice et Loi                 168
Politique et Géopolitique      168
Social                         168
Transport et Infrastructure    168
Économie                       168
Éducation et Jeunesse          167
Santé                            1
dtype: int64

In [35]:
df_variables.drop(columns="Santé", inplace=True)

# Encoding the values

In [36]:
# Function which transform "POSITIF" in 1, "NEUTRE" in 0, "NEGATIF" in "-1" 
def indicateur_transform(s):
    if s.lower()=="negatif":
        return -1
    if s.lower()=="positif":
        return 1
    return 0

# Function which use the openai API to encode a string 
def embed(string: str):
    if string=="":
        return np.NaN
    response = openai.Embedding.create(
        input=string,
        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']
    return embeddings

# encoding of every value in the dataframe
df_variables_embed = df_variables.copy().drop(columns="Indicateur")
df_variables_embed.fillna("", inplace=True)
df_variables_embed = df_variables_embed.applymap(embed)
df_variables_embed["Indicateur"] = df_variables["Indicateur"].apply(indicateur_transform)
df_variables_embed.to_csv('dataset_variable_embeded.csv', index=False)

df_variables_embed

,Condition de vie,Entreprise,Environnement,Identités et Personnalités,International,Justice et Loi,Politique et Géopolitique,Social,Transport et Infrastructure,Économie,Éducation et Jeunesse,Indicateur
0,"[0.007739204913377762, -0.01711335778236389, 0...","[0.008864439092576504, -0.008117960765957832, ...",NaN,"[-0.016569945961236954, -0.0011588307097554207...",NaN,"[-0.018591180443763733, -0.01260040607303381, ...","[-0.015994155779480934, -0.016831060871481895,...","[-0.003628625301644206, -0.017673850059509277,...","[0.0031558845657855272, -0.005453890189528465,...","[-0.0011510587064549327, -0.013634348288178444...","[-0.003864554688334465, -0.01885244995355606, ...",-1
1,"[-0.01586250774562359, -0.019095871597528458, ...",NaN,NaN,NaN,NaN,NaN,"[-0.011374819092452526, -0.02344169095158577, ...","[-0.01686009205877781, -0.022330019623041153, ...",NaN,"[-0.016873352229595184, -0.022343283519148827,...",NaN,1
2,NaN,"[-0.005539446137845516, -0.02231530472636223, ...","[0.015567711554467678, -0.022220579907298088, ...","[0.001817996148020029, 0.009034276008605957, 0...","[-0.014610902406275272, 0.00046385853784158826...","[-0.008008669130504131, -0.015978524461388588,...","[0.0049391379579901695, -0.024362051859498024,...","[0.00627171341329813, -0.03509081155061722, 0....",NaN,"[-0.0013075212482362986, -0.03191281855106354,...",NaN,-1
3,"[0.008480370044708252, 0.016012541949748993, 0...",NaN,NaN,"[-0.005596627481281757, -0.00791015662252903, ...",NaN,"[0.008250094950199127, -0.0027227280661463737,...","[-0.005028565414249897, -0.005247058812528849,...","[0.0195816308259964, 0.0169282928109169, 0.012...",NaN,"[-0.009089309722185135, 0.015599038451910019, ...",NaN,-1
4,"[-0.020505305379629135, -0.010142479091882706,...",NaN,NaN,"[-0.034629564732313156, -0.015229473821818829,...",NaN,NaN,"[-0.017641859129071236, -0.0261891707777977, 0...","[-0.008064118213951588, -0.007951565086841583,...",NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
163,"[-0.0036708845291286707, -0.000311443815007805...",NaN,NaN,"[-0.0007488129194825888, -0.005143261048942804...","[-0.0012931604869663715, 0.004211863502860069,...",NaN,"[-0.014549857005476952, 0.008568545803427696, ...","[-0.0036708845291286707, -0.000311443815007805...","[-0.0036708845291286707, -0.000311443815007805...",NaN,NaN,-1
164,"[-0.004038982558995485, -0.002825685078278184,...","[-0.002397802658379078, -0.030626336112618446,...","[-0.012616301886737347, -0.019392572343349457,...","[0.0012295192573219538, -0.01631734147667885, ...","[-0.003496744902804494, -0.019754674285650253,...",NaN,"[-0.0008030385943129659, -0.020906010642647743...",NaN,NaN,"[-0.0028103573713451624, -0.02640761248767376,...",NaN,0
165,NaN,"[-0.009980311617255211, -0.008039695210754871,...","[-0.009866082109510899, 0.007607816718518734, ...","[0.00628319475799799, 0.0047867135144770145, 0...","[-0.0008223910117521882, -0.011868895962834358...",NaN,"[-0.009995127096772194, -0.00804768968373537, ...",NaN,NaN,NaN,NaN,0
166,"[0.003862135112285614, -0.02650146558880806, 0...",NaN,"[-0.005819422192871571, -0.01554866787046194, ...","[-0.027279013767838478, -0.010332157835364342,...","[-0.014885477721691132, -0.017390435561537743,...","[-0.012618967331945896, -0.03240109235048294, ...","[-0.017374452203512192, -0.025708962231874466,...","[0.003845214145258069, -0.026542525738477707, ...",NaN,"[-0.006632940843701363, -0.04802752658724785, ...",NaN,0


# PCA reduction

In [39]:
# Transform vectors into reels: use of a PCA on each column but the "Indicateur" column which is already numeric
df_pca = df_variables_embed.copy()
for col in df_variables_embed.columns.drop("Indicateur"):
    _series = df_pca[col]
    _list_without_nan = list(_series[~_series.isna()])
    pca_coeff = pca_components_from_embeddings(_list_without_nan, 1)
    _series[~_series.isna()] = pca_coeff

df_pca.fillna(0, inplace=True)


col_abssum = df_pca.abs().sum()
df_pca = df_pca.loc[:,~(col_abssum==0)]
df_pca["Indicateur"] = df_variables_embed["Indicateur"]
df_pca.to_csv("dataset_pca.csv", index=False)

/var/folders/50/dt8t47812vngkqtnnpmfmzwc0000gn/T/ipykernel_79825/554281783.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _series[~_series.isna()] = pca_coeff
/var/folders/50/dt8t47812vngkqtnnpmfmzwc0000gn/T/ipykernel_79825/554281783.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _series[~_series.isna()] = pca_coeff
/var/folders/50/dt8t47812vngkqtnnpmfmzwc0000gn/T/ipykernel_79825/554281783.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [41]:
df_pca

,Condition de vie,Entreprise,Environnement,Identités et Personnalités,International,Justice et Loi,Politique et Géopolitique,Social,Transport et Infrastructure,Économie,Éducation et Jeunesse,Indicateur
0,0.100675,-0.126372,0.000000,-0.212632,0.000000,0.183638,-0.125077,-0.047720,0.077607,-0.116324,-0.029752,-1
1,0.076267,0.000000,0.000000,0.000000,0.000000,0.000000,-0.110879,-0.133311,0.000000,-0.024927,0.000000,1
2,0.000000,0.032990,-0.117907,-0.151045,0.141100,0.202525,-0.043930,-0.003697,0.000000,-0.028461,0.000000,-1
3,0.083990,0.000000,0.000000,-0.045589,0.000000,-0.098729,-0.190072,-0.020814,0.000000,-0.175170,0.000000,-1
4,-0.122396,0.000000,0.000000,-0.036841,0.000000,0.000000,-0.124132,-0.066578,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
163,-0.141130,0.000000,0.000000,-0.036841,-0.074721,0.000000,-0.133786,-0.115378,0.067193,0.000000,0.000000,-1
164,-0.011973,0.133418,-0.179681,-0.133020,-0.016352,0.000000,-0.112848,0.000000,0.000000,0.322248,0.000000,0
165,0.000000,0.279493,-0.161135,-0.015917,-0.066266,0.000000,-0.109355,0.000000,0.000000,0.000000,0.000000,0
166,0.035045,0.000000,0.132632,-0.003103,0.339691,-0.128431,-0.109946,0.038189,0.000000,0.020974,0.000000,0


# Causal AI

In [ ]:
# Creation of GIES model in order to find the causal relationship between 

model_gies = cdt.causality.graph.GIES()
graph_gies = model_gies.predict(df_pca, skeleton)

# visualize network
fig=plt.figure(figsize=(15,10))
nx.draw_networkx(graph_gies, font_size=18, font_color='r')

# get adjaceny matrix of GIES graph
A_gies = nx.adjacency_matrix(graph_gies).todense()

In [ ]:
import pandas as pd
import cdt
import networkx as nx
import matplotlib.pyplot as plt

cdt.SETTINGS.rpath = 'C:/Program Files/R/R-4.2.2/bin/Rscript' #à changer en fonction de l'emplacement de R 

# intialize graph lasso
glasso = cdt.independence.graph.Glasso()

# apply graph lasso to dataset
skeleton = glasso.predict(df)

# visualize network
fig=plt.figure(figsize=(15,10))
nx.draw_networkx(skeleton, font_size=18, font_color='r')